<a href="https://colab.research.google.com/github/Softmax20/hello-world/blob/master/find_point_by_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install lifelines

     |████████████████████████████████| 348kB 2.1MB/s 


In [2]:
from lifelines.statistics import logrank_test
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

# 구글 드라이브에서 대회 데이터셋 DataFrame으로 불러오기
from google.colab import drive
drive.mount("/content/gdrive")
%cd /content/gdrive/My Drive
X_df = pd.read_csv('trainX.csv')
Y_df = pd.read_csv('trainY.csv')
A_df = pd.read_csv('trainA.csv')
test_df = pd.read_csv('testX.csv')

tabels = [X_df, Y_df, A_df, test_df]

for tabel in tabels:
  tabel.drop('Unnamed: 0', axis=1, inplace=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive


In [3]:
from sklearn.neighbors import NearestNeighbors

In [11]:
# indices는 p<0.01의 군집 중심
indices = []
Time = Y_df['time']

for n in range(7, 11):
  # kneighbors: testX의 각 포인트를 중심으로 n개의 인접 환자
  neigh = NearestNeighbors(n_neighbors=n).fit(X_df)
  kneighbors = neigh.kneighbors(test_df, n_neighbors=n)[1]
  for i in range(0, 286):
    # Action에 따라 군집을 나누기 위해 인덱스 추출
    Action = A_df.iloc[kneighbors[i]]
    Action1 = Action[Action['Action']==1].index
    Action0 = Action[Action['Action']==0].index
    # log rank test 시행
    results = logrank_test(Time.iloc[Action1], Time.iloc[Action0], event_observed_A=Y_df['event'].iloc[Action1], event_observed_B=Y_df['event'].iloc[Action0])
    # p<0.01 일 경우 결과 출력 및 군집 중심 저장
    if results.p_value < 0.01:
      print(f'i={i}, n_neighbors={n}, p_value={results.p_value:.3f}')
      indices.append(i)

i=121, n_neighbors=7, p_value=0.008
i=179, n_neighbors=7, p_value=0.010
i=252, n_neighbors=7, p_value=0.008
i=272, n_neighbors=7, p_value=0.008
i=7, n_neighbors=8, p_value=0.008
i=18, n_neighbors=8, p_value=0.008
i=25, n_neighbors=8, p_value=0.008
i=35, n_neighbors=8, p_value=0.008
i=62, n_neighbors=8, p_value=0.007
i=69, n_neighbors=8, p_value=0.007
i=70, n_neighbors=8, p_value=0.004
i=76, n_neighbors=8, p_value=0.007
i=87, n_neighbors=8, p_value=0.008
i=94, n_neighbors=8, p_value=0.008
i=121, n_neighbors=8, p_value=0.004
i=144, n_neighbors=8, p_value=0.008
i=150, n_neighbors=8, p_value=0.008
i=179, n_neighbors=8, p_value=0.005
i=187, n_neighbors=8, p_value=0.008
i=212, n_neighbors=8, p_value=0.008
i=225, n_neighbors=8, p_value=0.008
i=252, n_neighbors=8, p_value=0.004
i=259, n_neighbors=8, p_value=0.008
i=265, n_neighbors=8, p_value=0.004
i=266, n_neighbors=8, p_value=0.008
i=270, n_neighbors=8, p_value=0.008
i=272, n_neighbors=8, p_value=0.004
i=280, n_neighbors=8, p_value=0.008
i=7

In [12]:
# 군집 중심 리스트
indices = list(set(indices))
print(indices)

[259, 7, 263, 265, 266, 135, 270, 272, 144, 18, 147, 150, 151, 280, 25, 35, 46, 179, 187, 62, 69, 70, 76, 81, 212, 87, 94, 225, 121, 252]


In [13]:
# 군집 중심 갯수
len(indices)

30